In [0]:
%python
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType, CharType
from pyspark.sql.functions import *

In [0]:
%python
delimeter= ','

airport_data= "/FileStore/tables/airports_mod.dat"
routes_data= "/FileStore/tables/routes.dat"
final_airlines="/FileStore/tables/Final_airlines"

In [0]:
%python
sc

SparkContext 

 Spark UI 

 
 Version 
 v3.3.0 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
%python
airport_schema=StructType([
    StructField('Airport_ID',       IntegerType(), nullable=True),
    StructField('Name',   StringType(), nullable=True),
    StructField('City',   StringType(), nullable=True),
    StructField('Country',   StringType(), nullable=True),
    StructField('IATA_FAA',   StringType(), nullable=True),
    StructField('ICAO',   StringType(), nullable=True),
    StructField('Latitude',   FloatType(), nullable=True),
    StructField('Longitude',   FloatType(), nullable=True),
    StructField('Altitude',   IntegerType(), nullable=True),
    StructField('Timezone', IntegerType(), nullable=True),
    StructField('DST',   StringType(), nullable=True),
    StructField('Tz',   StringType(), nullable=True),
 ])

In [0]:
%python
airport_df= spark.read.format("csv").option("sep",delimeter).schema(airport_schema).load(airport_data)
airport_df.count()

Out[10]: 8107

In [0]:
%python
airline_schema=StructType([
    StructField("Airline", StringType(), nullable=True),
    StructField("Name", StringType(), nullable=True),
    StructField("Alias", StringType(), nullable=True),
    StructField("IATA", StringType(), nullable=True),
    StructField("ICAO", StringType(), nullable=True),
    StructField("Callsign", StringType(), nullable=True),
    StructField("Country", StringType(), nullable=True),
    StructField("Active", StringType(), nullable=True),
])

In [0]:
%python
airline_df= spark.read.format("csv").option("sep",delimeter).schema(airline_schema).load(final_airlines)
airline_df.count()
airline_df.show(10)

+-------+--------------------+-----+----+----+--------------+--------------+------+
|Airline|                Name|Alias|IATA|ICAO|      Callsign|       Country|Active|
+-------+--------------------+-----+----+----+--------------+--------------+------+
|      1|      Private flight|   \N|   -| N/A|          null|          null|     Y|
|      2|         135 Airways|   \N|null| GNL|       GENERAL| United States|     N|
|      3|       1Time Airline|   \N|  1T| RNX|       NEXTIME|  South Africa|     Y|
|      4|2 Sqn No 1 Elemen...|   \N|null| WYT|          null|United Kingdom|     N|
|      5|     213 Flight Unit|   \N|null| TFU|          null|        Russia|     N|
|      6|223 Flight Unit S...|   \N|null| CHD|CHKALOVSK-AVIA|        Russia|     N|
|      7|   224th Flight Unit|   \N|null| TTF|    CARGO UNIT|        Russia|     N|
|      8|         247 Jet Ltd|   \N|null| TWF|  CLOUD RUNNER|United Kingdom|     N|
|      9|         3D Aviation|   \N|null| SEC|       SECUREX| United States|

In [0]:
%python
routes_schema=StructType([
    StructField("Airline" , StringType(), nullable=True),
        StructField("Airline_ID" , IntegerType(), nullable=True),
        StructField("Source_airport" , StringType(), nullable=True),
        StructField("Source_airport_ID" , IntegerType(), nullable=True),
        StructField("Destination_airport" , StringType(), nullable=True),
        StructField("Destination_airport_ID" , IntegerType(), nullable=True),
        StructField("Codeshare" , StringType(), nullable=True),
        StructField("Stops" , IntegerType(), nullable=True),
        StructField("Equipment" , StringType(), nullable=True),
])

In [0]:
%python
routes_df= spark.read.format("csv").option("sep",delimeter).schema(routes_schema).load(routes_data)
routes_df.count()
routes_df.show(10)

+-------+----------+--------------+-----------------+-------------------+----------------------+---------+-----+---------+
|Airline|Airline_ID|Source_airport|Source_airport_ID|Destination_airport|Destination_airport_ID|Codeshare|Stops|Equipment|
+-------+----------+--------------+-----------------+-------------------+----------------------+---------+-----+---------+
|     2B|       410|           AER|             2965|                KZN|                  2990|     null|    0|      CR2|
|     2B|       410|           ASF|             2966|                KZN|                  2990|     null|    0|      CR2|
|     2B|       410|           ASF|             2966|                MRV|                  2962|     null|    0|      CR2|
|     2B|       410|           CEK|             2968|                KZN|                  2990|     null|    0|      CR2|
|     2B|       410|           CEK|             2968|                OVB|                  4078|     null|    0|      CR2|
|     2B|       

In [0]:
%python
airports_india=airport_df.select('Airport_ID').where(upper(airport_df.Country)=='INDIA').count()
airports_india

Out[15]: 140

In [0]:
%python
airline_with_zero_stops=airline_df.join(routes_df, airline_df.Airline == routes_df.Airline_ID)\
                        .filter(routes_df.Stops==0)\
                        .select(countDistinct(airline_df.Name))
                        

                        
airline_with_zero_stops.show()

+--------------------+
|count(DISTINCT Name)|
+--------------------+
|                 540|
+--------------------+



In [0]:
%python
airlines_list=airline_df.join(routes_df, airline_df.Airline == routes_df.Airline_ID)\
                         .filter(routes_df.Codeshare.isNotNull())\
                         .select(countDistinct(airline_df.Name))

airlines_list.show()

+--------------------+
|count(DISTINCT Name)|
+--------------------+
|                 146|
+--------------------+



In [0]:
%python
highest_airport=airport_df.select('Country','Airport_ID')\
                .groupBy('Country')\
                .agg(countDistinct('Airport_ID').alias('Number of airport'))\
                .orderBy('Number of airport',ascending = False).limit(1)
highest_airport.show()               

+-------------+-----------------+
|      Country|Number of airport|
+-------------+-----------------+
|United States|             1697|
+-------------+-----------------+



In [0]:
%python
Active_Airlines= airline_df.select(countDistinct('Name'))\
                .filter((airline_df.Country== 'United States') & (airline_df.Active=='Y'))
Active_Airlines.count()

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
<command-1949916851411603> in <cell line: 1>()
----> 1 Active_Airlines= airline_df.select(countDistinct('Name'))\
      2                 .filter((airline_df.Country== 'United States') & (airline_df.Active=='Y'))
      3 Active_Airlines.count()

/databricks/spark/python/pyspark/instrumentation_utils.py in wrapper(*args, **kwargs)
     46             start = time.perf_counter()
     47             try:
---> 48                 res = func(*args, **kwargs)
     49                 logger.log_success(
     50                     module_name, class_name, function_name, time.perf_counter() - start, signature

/databricks/spark/python/pyspark/sql/dataframe.py in filter(self, condition)
   2163             jdf = self._jdf.filter(condition)
   2164         elif isinstance(condition, Column):
-> 2165             jdf = self._jdf.filter(condition._jc

In [0]:
%python
airline_df.show

Out[20]: <bound method DataFrame.show of DataFrame[Airline: string, Name: string, Alias: string, IATA: string, ICAO: string, Callsign: string, Country: string, Active: string]>

In [0]:
%sql
result = (airline_df
          .join(routes_df, airline_df.Airline == routes_df.Airline_ID )\
          .join(airport_df, routes_df.Source_airport_ID == airport.Airport_ID).alias(ar1)\
          .join(airport_df, routes.dest_airport_id == airport.airport_id).alias(ar2)\
          .where((trim(upper(ar1.country)) == "UNITED STATES") |\
                 (trim(upper(ar2.country)) == "UNITED STATES"))\
          .select("airlines_df.Name")\
          .distinct())

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
<command-1949916851411605> in <cell line: 1>()
      1 result = (airline_df
      2           .join(routes_df, airline_df.Airline == routes_df.Airline_ID )\
----> 3           .join(airport_df, routes_df.Source_airport_ID == airport_df.Airport_ID).alias(ar1)\
      4           .join(airport_df, routes.dest_airport_id == airport.airport_id).alias(ar2)\
      5           .where((trim(upper(ar1.country)) == "UNITED STATES") |\

NameError: name 'ar1' is not defined

In [0]:
%python
select distinct a.name from
airlines a join routes r
on
a.airline_id = r.airline_id

join

airport ar1
on
ar1.airport_id = r.src_airport_id

join 
airport ar2
on
ar2.airport_id = r.dest_airport_id

where

trim(upper(ar1.country)) = "UNITED STATES" or
trim(upper(ar2.country)) = "UNITED STATES"
;

In [0]:
%python
a=airline_df.alias('a')
r=routes_df.alias('r')
ar1=airport_df.alias('ar1')
ar2=airport_df.alias('ar2')

result = a.join(r, a.Airline == r.Airline_ID) \
          .join(airport_df.alias('ar1'), ar1.Airport_ID == r.Source_airport_ID )\
          .join(airport_df.alias('ar2'), ar2.Airport_ID == r.Destination_airport_ID)\
          .where((trim(upper(ar1.Country)) == "UNITED STATES") |\
                 (trim(upper(ar2.Country)) == "UNITED STATES"))\
          .select(a.Name).distinct()\
          .distinct()
display(result)

Name
Kenmore Air
EVA Air
Icelandair
Cathay Pacific
Ethiopian Airlines
Etihad Airways
T.J. Air
Alitalia
Philippine Airlines
Turkish Airlines


In [0]:
%python
result = airline_df.join(routes_df, (airline_df.Airline == routes_df.Airline_ID )\
          .join(airport_df.alias('airport_df'), airport_df.Airport_ID == routes_df.Source_airport_ID )\
          .join(airport_df.alias('airport_df'), airport_df.Airport_ID == routes_df.Destination_airport_ID)\
          .filter((trim(upper(airport_df.Country)) == "UNITED STATES") |(trim(upper(airport_df.Country)) == "UNITED STATES"))\
          .select(airline_df.Name).distinct()

  File "<command-4290303293607643>", line 5
    .select(airline_df.Name).distinct()
                                       
^
SyntaxError: unexpected EOF while parsing


In [0]:
%python
result = airline_df.alias('a').join(routes_df.alias('r'), col('a.Airline') == col('r.Airline_ID') )\
          .join(airport_df.alias('ar1'), col('ar1.Airport_ID') == col('r.Source_airport_ID') )\
          .join(airport_df.alias('ar2'), col('ar2.Airport_ID') == col('r.Destination_airport_ID'))\
          .filter((trim(upper('ar1.Country')) == "UNITED STATES") |(trim(upper('ar2.Country')) == "UNITED STATES"))\
          .select(airline_df.Name).distinct()
display(result)

Name
Kenmore Air
EVA Air
Icelandair
Cathay Pacific
Ethiopian Airlines
Etihad Airways
T.J. Air
Alitalia
Gulf Air Bahrain
Philippine Airlines


In [0]:
%sql
select distinct a.name from
airline_df a join routes_df r
on
a.airline_id = r.airline_id

join

airport_df ar1
on
ar1.airport_ID = r.Source_airport_ID

join 
airport_df ar2
on
ar2.airport_ID = r.Destination_airport_ID

where

trim(upper(ar1.country)) = "UNITED STATES" or
trim(upper(ar2.country)) = "UNITED STATES"
;


com.databricks.backend.common.rpc.SparkDriverExceptions$SQLExecutionException: org.apache.spark.sql.AnalysisException: Table or view not found: airline_df; line 2 pos 0;
'Distinct
+- 'Project ['a.name]
 +- 'Filter (('trim('upper('ar1.country)) = UNITED STATES) OR ('trim('upper('ar2.country)) = UNITED STATES))
 +- 'Join Inner, ('ar2.airport_ID = 'r.Destination_airport_ID)
 :- 'Join Inner, ('ar1.airport_ID = 'r.Source_airport_ID)
 : :- 'Join Inner, ('a.airline_id = 'r.airline_id)
 : : :- 'SubqueryAlias a
 : : : +- 'UnresolvedRelation [airline_df], [], false
 : : +- 'SubqueryAlias r
 : : +- 'UnresolvedRelation [routes_df], [], false
 : +- 'SubqueryAlias ar1
 : +- 'UnresolvedRelation [airport_df], [], false
 +- 'SubqueryAlias ar2
 +- 'UnresolvedRelation [airport_df], [], false

	at org.apache.spark.sql.catalyst.analysis.package$AnalysisErrorAt.failAnalysis(package.scala:45)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.$anonfun$checkAnalysis$2(CheckAnalysis.scala:158)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.$anonfun$checkAnalysis$2$adapted(CheckAnalysis.scala:125)
	at org.apache.spark.sql.catalyst.trees.TreeNode.foreachUp(TreeNode.scala:295)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$foreachUp$1(TreeNode.scala:294)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$foreachUp$1$adapted(TreeNode.scala:294)
	at scala.collection.Iterator.foreach(Iterator.scala:943)
	at scala.collection.Iterator.foreach$(Iterator.scala:943)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1431)
	at scala.collection.IterableLike.foreach(IterableLike.scala:74)
	at scala.collection.IterableLike.foreach$(IterableLike.scala:73)
	at scala.collection.AbstractIterable.foreach(Iterable.scala:56)
	at org.apache.spark.sql.catalyst.trees.TreeNode.foreachUp(TreeNode.scala:294)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$foreachUp$1(TreeNode.scala:294)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$foreachUp$1$adapted(TreeNode.scala:294)
	at scala.collection.Iterator.foreach(Iterator.scala:943)
	at scala.collection.Iterator.foreach$(Iterator.scala:943)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1431)
	at scala.collection.IterableLike.foreach(IterableLike.scala:74)
	at scala.collection.IterableLike.foreach$(IterableLike.scala:73)
	at scala.collection.AbstractIterable.foreach(Iterable.scala:56)
	at org.apache.spark.sql.catalyst.trees.TreeNode.foreachUp(TreeNode.scala:294)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$foreachUp$1(TreeNode.scala:294)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$foreachUp$1$adapted(TreeNode.scala:294)
	at scala.collection.Iterator.foreach(Iterator.scala:943)
	at scala.collection.Iterator.foreach$(Iterator.scala:943)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1431)
	at scala.collection.IterableLike.foreach(IterableLike.scala:74)
	at scala.collection.IterableLike.foreach$(IterableLike.scala:73)
	at scala.collection.AbstractIterable.foreach(Iterable.scala:56)
	at org.apache.spark.sql.catalyst.trees.TreeNode.foreachUp(TreeNode.scala:294)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$foreachUp$1(TreeNode.scala:294)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$foreachUp$1$adapted(TreeNode.scala:294)
	at scala.collection.Iterator.foreach(Iterator.scala:943)
	at scala.collection.Iterator.foreach$(Iterator.scala:943)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1431)
	at scala.collection.IterableLike.foreach(IterableLike.scala:74)
	at scala.collection.IterableLike.foreach$(IterableLike.scala:73)
	at scala.collection.AbstractIterable.foreach(Iterable.scala:56)
	at org.apache.spark.sql.catalyst.trees.TreeNode.foreachUp(TreeNode.scala:294)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$foreachUp$1(TreeNode.scala:294)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$foreachUp$1$adapted(TreeNode.scala:294)
	at scala.collection.Iterator.foreach(It

In [0]:
airline_df.registerTempTable("airlines")
routes_df.registerTempTable("routes")
airport_df.registerTempTable("airport")

In [0]:
result=spark.sql("select distinct a.name from airlines a join routes r on a.Airline = r.Airline_ID \
join airport ar1 on ar1.airport_ID = r.Source_airport_ID join airport ar2 on ar2.Airport_ID = r.Destination_airport_ID \
where ar1.country = 'United States' or ar2.country = 'United States';")
result.count()

Out[36]: 136

In [0]:
sc

SparkContext 

 Spark UI 

 
 Version 
 v3.3.0 
 Master 
 local[8] 
 AppName 
 Databricks Shell